# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up graphlab create

In [1]:
import graphlab

A newer version of GraphLab Create (v1.10) is available! Your current version is v1.9.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

2016-06-06 23:15:41,684 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: /tmp/graphlab_server_1465269339.log


This non-commercial license of GraphLab Create is assigned to lydiahuo0918@gamil.com and will expire on April 13, 2017. For commercial licensing options, visit https://dato.com/buy/.


# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [3]:
train_data,test_data = sales.random_split(.8,seed=0)

In [6]:
sales.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650,1,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242,2,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000,1,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000,1,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080,1,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930,1,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819,2,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711,1,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470,1,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560,2,0


In [33]:
sales['bedrooms_squared'] = sales['bedrooms'] * sales['bedrooms']
sales['bed_bath_rooms'] = sales['bedrooms'] * sales['bathrooms']
sales['log_sqft_living'] = sales['sqft_living'].apply(lambda x: math.log(x))
sales['lat_plus_long'] = sales['lat'] + sales['long']

In [34]:
sales.head()
train_data,test_data = sales.random_split(.8,seed=0)

In [35]:
print test_data['bedrooms_squared'].mean()
print test_data['bed_bath_rooms'].mean()
print test_data['log_sqft_living'].mean()
print test_data['lat_plus_long'].mean()

12.4466777016
7.50390163159
7.55027467965
-74.6533349722


In [21]:
print type((sales['sqft_living']))

<class 'graphlab.data_structures.sarray.SArray'>


id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650,1,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242,2,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000,1,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000,1,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080,1,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930,1,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819,2,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711,1,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470,1,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560,2,0


In [43]:
Model1 = graphlab.linear_regression.create(train_data, target='price',
                                           features=['sqft_living','bedrooms','bathrooms','lat','long'])

Linear regression:

--------------------------------------------------------

Number of examples          : 16537

Number of features          : 5

Number of unpacked features : 5

...

+-----------+----------+--------------+--------------------+----------------------+---------------+-----------------+

| Iteration | Passes   | Elapsed Time | Training-max_error | Validation-max_error | Training-rmse | Validation-rmse |

+-----------+----------+--------------+--------------------+----------------------+---------------+-----------------+

| 1         | 2        | 0.051930     | 4072893.256572     | 2047202.291304       | 237215.792423 | 220086.476912   |

+-----------+----------+--------------+--------------------+----------------------+---------------+-----------------+

In [42]:
Model2 = graphlab.linear_regression.create(train_data, target='price',
                                           features=['sqft_living','bedrooms','bathrooms','lat','long','bed_bath_rooms'])

Linear regression:

--------------------------------------------------------

Number of examples          : 16496

Number of features          : 6

Number of unpacked features : 6

...

+-----------+----------+--------------+--------------------+----------------------+---------------+-----------------+

| Iteration | Passes   | Elapsed Time | Training-max_error | Validation-max_error | Training-rmse | Validation-rmse |

+-----------+----------+--------------+--------------------+----------------------+---------------+-----------------+

| 1         | 2        | 0.071389     | 4015093.592968     | 1739096.525019       | 235457.160902 | 231035.405199   |

+-----------+----------+--------------+--------------------+----------------------+---------------+-----------------+

In [41]:
Model3 = graphlab.linear_regression.create(train_data, target='price',
                                           features=['sqft_living','bedrooms','bathrooms','lat','long','bed_bath_rooms','bedrooms_squared','log_sqft_living','lat_plus_long'])

Linear regression:

--------------------------------------------------------

Number of examples          : 16529

Number of features          : 9

Number of unpacked features : 9

...

+-----------+----------+--------------+--------------------+----------------------+---------------+-----------------+

| 1         | 2        | 0.049389     | 3170443.355384     | 2090946.995423       | 228612.054594 | 219944.757260   |

+-----------+----------+--------------+--------------------+----------------------+---------------+-----------------+

SUCCESS: Optimal solution found.

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



In [46]:
print Model1

Class                         : LinearRegression

Schema
------
Number of coefficients        : 6
Number of examples            : 16537
Number of feature columns     : 5
Number of unpacked features   : 5

Hyperparameters
---------------
L1 penalty                    : 0.0
L2 penalty                    : 0.01

Training Summary
----------------
Solver                        : newton
Solver iterations             : 1
Solver status                 : SUCCESS: Optimal solution found.
Training time (sec)           : 0.0817

Settings
--------
Residual sum of squares       : 9.30559020178e+14
Training RMSE                 : 237215.7924

Highest Positive Coefficients
-----------------------------
lat                           : 627799.0679
bathrooms                     : 13206.659
sqft_living                   : 310.7531

Lowest Negative Coefficients
----------------------------
(intercept)                   : -55686334.1451
long                          : -211865.0906
bedrooms                  

In [47]:
print Model2

Class                         : LinearRegression

Schema
------
Number of coefficients        : 7
Number of examples            : 16496
Number of feature columns     : 6
Number of unpacked features   : 6

Hyperparameters
---------------
L1 penalty                    : 0.0
L2 penalty                    : 0.01

Training Summary
----------------
Solver                        : newton
Solver iterations             : 1
Solver status                 : SUCCESS: Optimal solution found.
Training time (sec)           : 0.1042

Settings
--------
Residual sum of squares       : 9.14539470929e+14
Training RMSE                 : 235457.1609

Highest Positive Coefficients
-----------------------------
lat                           : 619868.1025
bed_bath_rooms                : 26980.8377
sqft_living                   : 304.6212

Lowest Negative Coefficients
----------------------------
(intercept)                   : -53803750.4155
long                          : -201166.8723
bedrooms                 

In [48]:
print Model3

Class                         : LinearRegression

Schema
------
Number of coefficients        : 10
Number of examples            : 16529
Number of feature columns     : 9
Number of unpacked features   : 9

Hyperparameters
---------------
L1 penalty                    : 0.0
L2 penalty                    : 0.01

Training Summary
----------------
Solver                        : newton
Solver iterations             : 1
Solver status                 : SUCCESS: Optimal solution found.
Training time (sec)           : 0.0751

Settings
--------
Residual sum of squares       : 8.63862920519e+14
Training RMSE                 : 228612.0546

Highest Positive Coefficients
-----------------------------
lat                           : 706957.3679
bathrooms                     : 73474.3971
bedrooms                      : 28462.9914
sqft_living                   : 534.7215

Lowest Negative Coefficients
----------------------------
(intercept)                   : -53192182.9585
log_sqft_living           

In [49]:
Model1 = graphlab.linear_regression.create(test_data, target='price',
                                           features=['sqft_living','bedrooms','bathrooms','lat','long'])
Model2 = graphlab.linear_regression.create(test_data, target='price',
                                           features=['sqft_living','bedrooms','bathrooms','lat','long','bed_bath_rooms'])
Model3 = graphlab.linear_regression.create(test_data, target='price',
                                           features=['sqft_living','bedrooms','bathrooms','lat','long','bed_bath_rooms','bedrooms_squared','log_sqft_living','lat_plus_long'])

Linear regression:

--------------------------------------------------------

Number of examples          : 4029

Number of features          : 5

Number of unpacked features : 5

...

+-----------+----------+--------------+--------------------+----------------------+---------------+-----------------+

| Iteration | Passes   | Elapsed Time | Training-max_error | Validation-max_error | Training-rmse | Validation-rmse |

+-----------+----------+--------------+--------------------+----------------------+---------------+-----------------+

| 1         | 2        | 0.019467     | 3175081.985339     | 1392247.300283       | 227193.204704 | 200624.102564   |

+-----------+----------+--------------+--------------------+----------------------+---------------+-----------------+

In [56]:
print Model1

Class                         : LinearRegression

Schema
------
Number of coefficients        : 6
Number of examples            : 4029
Number of feature columns     : 5
Number of unpacked features   : 5

Hyperparameters
---------------
L1 penalty                    : 0.0
L2 penalty                    : 0.01

Training Summary
----------------
Solver                        : newton
Solver iterations             : 1
Solver status                 : SUCCESS: Optimal solution found.
Training time (sec)           : 0.0212

Settings
--------
Residual sum of squares       : 2.17769447636e+14
Training RMSE                 : 232487.6251

Highest Positive Coefficients
-----------------------------
lat                           : 534929.7923
bathrooms                     : 19051.1425
sqft_living                   : 284.5401

Lowest Negative Coefficients
----------------------------
(intercept)                   : -39216439.1945
long                          : -112922.1544
bedrooms                  

In [57]:
print Model2

Class                         : LinearRegression

Schema
------
Number of coefficients        : 7
Number of examples            : 4005
Number of feature columns     : 6
Number of unpacked features   : 6

Hyperparameters
---------------
L1 penalty                    : 0.0
L2 penalty                    : 0.01

Training Summary
----------------
Solver                        : newton
Solver iterations             : 1
Solver status                 : SUCCESS: Optimal solution found.
Training time (sec)           : 0.0226

Settings
--------
Residual sum of squares       : 2.15888669627e+14
Training RMSE                 : 232174.0434

Highest Positive Coefficients
-----------------------------
lat                           : 522691.7934
bed_bath_rooms                : 32409.5568
sqft_living                   : 279.7047

Lowest Negative Coefficients
----------------------------
(intercept)                   : -37748844.9576
long                          : -107505.7674
bedrooms                  

In [58]:
print Model3

Class                         : LinearRegression

Schema
------
Number of coefficients        : 10
Number of examples            : 4006
Number of feature columns     : 9
Number of unpacked features   : 9

Hyperparameters
---------------
L1 penalty                    : 0.0
L2 penalty                    : 0.01

Training Summary
----------------
Solver                        : newton
Solver iterations             : 1
Solver status                 : SUCCESS: Optimal solution found.
Training time (sec)           : 0.0316

Settings
--------
Residual sum of squares       : 2.06776709568e+14
Training RMSE                 : 227193.2047

Highest Positive Coefficients
-----------------------------
lat                           : 534309.6879
bathrooms                     : 36315.155
bed_bath_rooms                : 1190.4493
bedrooms_squared              : 1139.7522
sqft_living                   : 472.7597

Lowest Negative Coefficients
----------------------------
(intercept)                   : -3

In [51]:
test_data['bedrooms_squared'].mean()

12.44667770158429

In [52]:
test_data['bed_bath_rooms'].mean()

7.503901631591395

In [53]:
test_data['log_sqft_living'].mean()

7.55027467964594

In [55]:
test_data['lat_plus_long'].mean()

-74.65333497217306

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [ ]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = graphlab.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                  validation_set = None)

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [ ]:
example_weight_summary = example_model.get("coefficients")
print example_weight_summary

# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [ ]:
example_predictions = example_model.predict(train_data)
print example_predictions[0] # should be 271789.505878

# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [ ]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions

    # Then compute the residuals/errors

    # Then square and add them up

    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [ ]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print rss_example_train # should be 2.7376153833e+14

# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [ ]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [ ]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [ ]:
# create the remaining 3 features in both TEST and TRAIN data



* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [ ]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [ ]:
# Learn the three models: (don't forget to set validation_set = None)


In [ ]:
# Examine/extract each model's coefficients:


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [ ]:
# Compute the RSS on TRAINING data for each of the three models and record the values:


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [ ]:
# Compute the RSS on TESTING data for each of the three models and record the values:


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected?Think about the features that were added to each model from the previous.